In [13]:
import openai, json, requests

client = openai.OpenAI()
messages = []

# ============================================================
# 1. Function Definitions (actual API calls)
# ============================================================
BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"

def get_popular_movies():
    """Fetch popular movies from /movies endpoint."""
    response = requests.get(f"{BASE_URL}/movies")
    response.raise_for_status()
    return json.dumps(response.json())


def get_movie_details(id):
    """Fetch movie details from /movies/:id endpoint."""
    response = requests.get(f"{BASE_URL}/movies/{id}")
    response.raise_for_status()
    return json.dumps(response.json())


def get_movie_credits(id):
    """Fetch cast & crew from /movies/:id/credits endpoint."""
    response = requests.get(f"{BASE_URL}/movies/{id}/credits")
    response.raise_for_status()
    return json.dumps(response.json())


FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_movie_credits": get_movie_credits,
}

In [14]:
# ============================================================
# 2. Tool (function) schemas — tells the LLM what it can call
# ============================================================
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get a list of currently popular movies. Call this when the user asks about trending or popular movies.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get detailed information about a specific movie by its ID. Call this when the user asks about a specific movie's details such as title, overview, release date, rating, etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The numeric ID of the movie.",
                    },
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "Get the cast and crew of a specific movie by its ID. Call this when the user asks who starred in or worked on a movie.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The numeric ID of the movie.",
                    },
                },
                "required": ["id"],
            },
        },
    },
]


In [15]:
# ============================================================
# 3. Process AI response — same recursive pattern as your code
# ============================================================
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    # --- Branch A: LLM wants to call one or more functions ---
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with arguments {arguments}")

            # Parse arguments JSON string → Python dict
            try:
                parsed_args = json.loads(arguments)
            except json.JSONDecodeError:
                parsed_args = {}

            function_to_run = FUNCTION_MAP.get(function_name)
            result = function_to_run(**parsed_args)

            print(f"Ran {function_name} with args {arguments} for results of {result}")

            # Record the tool result into messages (memory)
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        # Recurse — let the LLM read the tool results and respond
        call_ai()

    # --- Branch B: LLM responds with plain text (no tool call) ---
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")



In [16]:
# ============================================================
# 4. Call the LLM
# ============================================================
def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)


In [17]:
# ============================================================
# 5. Main loop
# ============================================================
print("=== Movie Expert Agent ===")
print('Type your question (or "q" to quit)\n')

while True:
    user_input = input("You: ")
    if user_input.lower() in ("quit", "q"):
        break

    messages.append({"role": "user", "content": user_input})
    print(f"User: {user_input}")
    call_ai()

=== Movie Expert Agent ===
Type your question (or "q" to quit)

User: What is most popular movie right now
Calling function: get_popular_movies with arguments {}
Ran get_popular_movies with args {} for results of [{"adult": false, "backdrop_path": "https://image.tmdb.org/t/p/w1280/7HKpc11uQfxnw0Y8tRUYn1fsKqE.jpg", "genre_ids": [878, 28, 53], "id": 1236153, "original_language": "en", "original_title": "Mercy", "overview": "In the near future, a detective stands on trial accused of murdering his wife. He has ninety minutes to prove his innocence to the advanced AI Judge he once championed, before it determines his fate.", "popularity": 649.4032, "poster_path": "https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg", "release_date": "2026-01-20", "title": "Mercy", "video": false, "vote_average": 7.1, "vote_count": 458}, {"adult": false, "backdrop_path": "https://image.tmdb.org/t/p/w1280/hHDNOlATHhre4eZ7aYz5cdyJLik.jpg", "genre_ids": [27, 53, 878], "id": 1272837, "original_langua